In [1]:
import argparse
import logging
import math
import os
import random
import shutil
from pathlib import Path
# import sys
# sys.path.append("../..")

import accelerate
import datasets
import numpy as np
import torch
import torch.nn.functional as F
import torch.autograd.functional as A
import torch.utils.checkpoint
import transformers
from accelerate import Accelerator
from accelerate.logging import get_logger
from accelerate.state import AcceleratorState
from accelerate.utils import ProjectConfiguration, set_seed
from datasets import load_dataset
from huggingface_hub import create_repo, upload_folder
from packaging import version
from PIL import Image
from torchvision import transforms
from tqdm.auto import tqdm
from transformers import CLIPTextModel, CLIPTokenizer
from transformers.utils import ContextManagers

import diffusers
from diffusers import AutoencoderKL, DDPMScheduler, StableDiffusionPipeline, UNet2DConditionModel
from diffusers.optimization import get_scheduler
from diffusers.training_utils import EMAModel
from diffusers.utils import check_min_version, deprecate, is_wandb_available
from diffusers.utils.import_utils import is_xformers_available

from examples.text_to_image.eval import compute_fid, compute_ppl
def deepspeed_zero_init_disabled_context_manager():
        """
        returns either a context list that includes one that will disable zero.Init or an empty context list
        """
        deepspeed_plugin = AcceleratorState().deepspeed_plugin if accelerate.state.is_initialized() else None
        if deepspeed_plugin is None:
            return []

        return [deepspeed_plugin.zero3_init_context_manager(enable=False)]
from accelerate import Accelerator
from accelerate.logging import get_logger
from accelerate.state import AcceleratorState
from accelerate.utils import ProjectConfiguration, set_seed
accelerator = Accelerator(
        gradient_accumulation_steps=1,
        mixed_precision="fp16"
    )
with ContextManagers(deepspeed_zero_init_disabled_context_manager()):
    text_encoder = CLIPTextModel.from_pretrained(
        "stabilityai/stable-diffusion-2", subfolder="text_encoder", revision=None
    )
    vae = AutoencoderKL.from_pretrained(
        "stabilityai/stable-diffusion-2", subfolder="vae", revision=None
    )

unet = UNet2DConditionModel.from_pretrained(
    "stabilityai/stable-diffusion-2", subfolder="unet", revision=None
)
tokenizer = CLIPTokenizer.from_pretrained(
       "stabilityai/stable-diffusion-2", subfolder="tokenizer", revision=None
    )

ModuleNotFoundError: No module named 'dnnlib'